# Notebook to PDF (as a document)

`jupyter-nbconvert written_report.ipynb --to pdf -TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}' --template template.tplx`

# Notebook to slide deck

`jupyter-nbconvert presentation.ipynb --to slides --TagRemovePreprocessor.remove_input_tags={\"remove-input\"} --post serve --ServePostProcessor.port=9997 --ServePostProcessor.ip="0.0.0.0"`

http://72.179.3.141:9997/nov8.slides.html

To convert to pdf, add ?print-pdf to URL, i.e.

http://72.179.3.141:9997/nov8.slides.html?print-pdf

Coverage of topics:

## Dan

part one:
* Background
* Time-frequency analysis
* Time - frequency tilings
* Constant-Q 
* perfect reconstruction
* Existing transforms and software
  * STFT
  * mel-filterbanks using FFT
  * Discrete wavelet transforms
* common techniques do not have perfect reconstruction
  * Either have uniform frequency resolution or perfectly constant-Q
    * uniform resolution leads to poor time resolution for high frequencies
    * uniform resolution leads to poor frequency resolution for low frequencies
    * perfectly constant-Q transforms lead to data structure which is impossible to work with.
  * mel filterbanks try to provide a compromise in resolution, but in doing so introduces new problems.
    * Undersampled for some frequency bands
    * oversampled for others
  * FFT based methods require real and imaginary.
    * Leads to phase reconstruction problem.
    * End up making more work for yourself by having to do griffin-lim or similar algorithm to solve problem which should not have been there in the first place.
* Size of data
* Compression and storage

## Utsha
* Haar filter (1min)
  * Simplest, dyadic filter
  * Perfect reconstruction
  * We are using as our building block
* Discussion about GPU & Convolution (2min)
  * What is Convolution? (Toeplitz, etc)
  * Sequential vs Parallel Convolution
  * GPU's analysis and bottlenecks
* Our algorithm (3min)
  * Dyadic Analysis & Sysnthesis Filterbank
  * Convolution as a building block
  * Decimation & Polyphase filters to utilize convolution
  * Cascading unbalanced & balanced binary tree structure
* Complexity analysis (1min)
* Experiment Results (2min)

## Dan

* Denoising demo

* Drawbacks of our method
  * not shift invariant
    * can be fixed by dual-tree complex wavelet transform
    * effectively oversamples by two
  * We used Haar filters (two point sum and difference).
    * Frequency selectivity may not be as good
  * Not fully constant-Q
    * makes some analysis much easier, but others might be more difficult
    * example: music transcription

# GPU Filter Banks for Audio

# Abstract

In our project, we implement a class of perfect-reconstruction filterbanks for audio analysis. We exploit parallelism across CPU threads for part of our algorithm and use a GPU to solve embarassingly parallel subtasks. We also introduce a novel type of time-frequency tiling which retains the benefits of constant-Q transforms without requiring their cumbersome data structure.

# Background

For audio signal processing problems such as compression, denoising, and various types of pattern recognition, deep learning has taken over. For example, the search interest for "wavelet" has followed the opposite trajectory as "convolutional neural network"

![](img/trends1.png)

A current trend of audio signal processing research is to apply 2D convolutional neural networks to time-frequency representations of audio. Although great progress was made in the pre-deep learning era on perfect reconstruction filterbanks and wavelet transforms, most of the algorithms and techniques have been abandoned by practitioners of deep learning because no standardized and free implementations were ever widely disseminated. As a result, the standarized set of triangular "Mel" filterbanks dating back nearly 50 years appears to have remained the dominant tool, and have even seen a resurgance despite their [impending obsolescence.][1]

![](img/trends2.png)

The use of these methods is accumulating a considerable debt. Compared to the most recent iterations of constant-Q filterbanks, these methods for time-frequency analysis lack several desirable properties

* Parameterization/tunability
* Perfect reconstruction
* Meaningful/interpretable phase representation
* Generalization to multiple channels/phased arrays
* Efficient implementation for oversampled representations.

[1]:https://asmp-eurasipjournals.springeropen.com/articles/10.1186/s13636-021-00217-4

# Filter Banks and Wavelet Transforms

Filter banks are the oldest method of time-frequency analysis, dating back to the first analog spectrum analyzer built by Hermann von Helmholtz in the 19th century.

![](img/helmholtz.png)

In 1909, mathematician Alfréd Haar discovered that a list of $2^n$ numbers can be represented by recursively taking the sum and difference of adjacent pairs. This property is now commonly referred to 'alias cancellation'

In the 1970s, engineers created the first discrete, invertible filterbanks by expoiting this property. Using what are known as 'conjugate mirror filters'. In the following decade, a massive research effort took place, resulting in a rigorous mathematical understanding of these processes and the development of various "wavelet transforms."

Unfortunately, deep learning has begun to displace these techniques in education, leading to considerable confusion and technical debt.

In [ ]:
using DSP

# Haar Decomposition

(Draft) Haar decomposition is one of the simplest wavelet transform that has perfect reconstruction property, thus is a good introduction example. First of all, note that the input is a sequence of integers, because we are dealing digital data after sampling from raw data, such as audio. By performing two convolutions with the input, we get a low-frequency component and a high-frequency component.
As we can see, the sum and difference of the high and low frequency components forms a splitted signal of the original signal, which we can merge to retain the original one. This is an example of “critically sampled” dyadic filterbank, which means that the size of data after applying the filter is exactly same as original data. Note that, “dyadic filters” refers “dynamic analysis filters”, the type of filtering that consists of a filtering followed by downsampling. There is an inverse operation, which is synthetic filters. (Which we are dealing by convolution. We will get to this detail later)

In [ ]:
x = round.(Int,10*randn(8));

In [ ]:
print(x)

[7, 1, -13, 20, 4, 7, -18, 5]

In [ ]:
s = conv(x,[1, 1])[2:2:end];
d = conv(x,[1,-1])[2:2:end];

In [ ]:
println("s = ",s); print("d = ",d)

s = [8, 7, 11, -13]
d = [-6, 33, 3, 23]

In [ ]:
y1 = (s-d)/2;
y2 = (s+d)/2;

In [ ]:
println(Int.(y1)); print(Int.(y2))

[7, -13, 4, -18]
[1, 20, 7, 5]

This is an example of a **critically sampled** dyadic filterbank

# Time-frequency tiling



Decomposing the lower branch produces octave filters

img/dyadic_analysis_filterbank.svg

![](img/octave_tiling.png)


Decomposing both branches (balanced tree)

(uniform grid of rectangles) 

you can have anything in the middle

(Draft)
In the Haar decomposition, we received two signals with half time resolution but twice the frequency resolution signals, by applying the filters. Next, we introduce a more fine-grained time-frequency tiling pattern, as shown in the image.

This structure has the benefit of a simple data structure, since each tile is a vector, while providing the exact trade-off between the frequency resolution and the time resolution as we want.

# DWT Sequential time complexity

each level is N log N

log N levels

between N log N and N (log N)^2

regardless of how many levels you go down

(Draft) In general, we have between $N\log N$ and $N (\log N)^2$ time complexity to perform any discrete wavelet transform, because at each level of filtering tree, it takes $N \log N$ time and there are $\log N$ levels.

# parallel fft complexity (and therefore convolution)

log N levels

each level n/p computation

(N/p) log N

parallel DWT is

(N/p) (log N)^2

(Draft) Parallel FFT (Fast Fourier Transform) is widely used in any signal processing application. We can see FFT as a version of filterbank, which breaks the input signal down all the way to full frequency resolution, but with no time resolution. 

Similar to general DWT, we can see FFT consisting of $\log N$ levels. Each level has $\frac{N}{p}$ computation
, thus time complexity for parallel FFT is $(\frac{N}{p})\log N$. In contrast, parallel DWT takes $(\frac{N}{p})(\log N)^2$ time.

# Input size

Speech: few seconds long, fs = 16000, N ~ 10^5 per record.

Music: minutes long, two channels, fs = 48000, N ~ 10^7 per record.

Passive sonar array: 100s of channels, fs in the kHz, ~ 10^10 per hour.

# Data pipeline

Speech and music are nearly always compressed. Typical compression ratio is 10:1

Phased array data often minimally compressed and stored on cheap magnetic tape.

Often helpful to combine PRAM and GPU computation models.

Multiple CPUs can increase the rate at which data is decoded and communicated to the GPU(s)

# Generalization to other filters

You need alias cancellation

https://en.wikipedia.org/wiki/Quadrature_mirror_filter

As long as you satisfy this condition, you can use better filters

(Draft) The framework to process the input signal by the analysis filterbank and then reconstruct to get it back is generalizable. We can apply any operation on the in-between layer (i.e. the processed small data blocks) and retain the perfect reconstruction property, as long as the operation has mirror-conjugate property. 

# Discrete wavelet transform

Generalizes this idea 

# Goal

Generalize these types of transformations to

* Filters with greater frequency selectivity
* Oversampled filters
* Undersampled filters

# GPU convolution

what is CUDA doing?

https://docs.nvidia.com/deeplearning/cudnn/developer-guide/index.html

If you repeat your data and construct 

https://en.wikipedia.org/wiki/Toeplitz_matrix

Then convolution is equivalent to matrix multiply

Not actually doing matrix multiplication, but create a mapping from "virtual matrix" to corresponding array location

(fold in some of the theory from lecture on mat mul)

(Draft) We utilized existing implementation of CuDNN convolution, as it is optimized for GPU as shipped. The convolution on digital signal can be expressed as a multiplication of Toepliz matrix and the convolution kernel - whose rows consists of a shifted element of the original signal (i.e. sequence of numbers).
Note that Toepliz matrix multiplication is much efficient than regular matrix multiplication, because in implementation we do not need to store data in a matrix. Instead, it can be operated with indices and compute convolution, by creating a mapping from “virtual matrix” to the corresponding array location.

convolution is identical to mat vec multiply $Ax$ where $A$ is $N \times (N-M+\text{pad})$ toeplitz and $x$ is $(N-M+\text{pad})\times 1$

$\text{pad} \in [0,M]$ is the amount of zero padding

$N$ is signal length

$M$ is filter length

Sequential algorithm for convolution:

$(N-M+\text{pad})$ dot products.  Each dot product takes $M^2$ multiplies  and $M-1$ additions.

$ y = x \ast h = 
\begin{bmatrix}
    x_1 & 0 & \cdots & 0 & 0 \\
    x_2 & x_1 &      & \vdots & \vdots \\
    x_3 & x_2 & \cdots & 0 & 0 \\
    \vdots & x_3 & \cdots & x_1 & 0 \\
    x_{n-1} & \vdots & \ddots & x_2 & x_1 \\
    x_n & x_{n-1} &      & \vdots & x_2 \\
    0 & x_n & \ddots & x_{n-2} & \vdots \\
    0 & 0 & \cdots & x_{n-1} & x_{n-2} \\
    \vdots & \vdots &        & x_n & x_{n-1} \\
    0 & 0 & 0 & \cdots & x_n
\end{bmatrix}
$
$\begin{bmatrix}
    0 \\
    0 \\
    \vdots\\
    h_1 \\
    h_2 \\
    \vdots \\
    h_m \\
    0 \\
    \vdots \\
    0 \\
\end{bmatrix}$

## parallel convolution

Option 1: number of available processors is approximately N:

Use $p=(N-M+\text{pad})$ processors, Each processor does the dot product sequentially using $M^2$ multiplies and $M-1$ additions.

Second option: number of available processors is $\geq NM$

use $NM$ processors to perform all $N$ Hadamard products in parallel.

Use parallel reduce to compute the sum in $log_2(M)$ steps

### Question: how many $p$ would we need for audio signals?

Speech: few seconds long, fs = 16000, N ~ 10^5 per record.

Music: minutes long, two channels, fs = 48000, N ~ 10^7 per record.

Passive sonar array: 100s of channels, fs in the kHz, ~ 10^10 per hour.

### Ideally, we would have 10^10 processors.

In practice N is limited by the ability of CPU/ data storage to stream data onto the GPU.

GPU clock rate is typically about 1 Ghz. There is $\approx 1 \text{ns}$ between clock cycles on GPU. N is chosen based on how much data can be loaded into memory in that period of time.

For small M, convolution is effectively $O(1)$ on GPU, but is limited by memory transfer rate.

Deep learning practitioners have highly optimized this problem (convolution between large N and small M). We want to build our algorithm on top of their software and highly optimized data pipeline.

Exactly the type of thing that GPUs are built for.

Use gpus which act like $p = O(10^3)$ processors. Break up data so that $NM$ words of data can be stored on the gpu.

often bottlenecked by ability of CPU to decode compressed data.

Really just limited by ability of CPU to transfer memory back and forth.



# Fundamental building block of our algorithm:

Highly optimized CuDNN convolution routine which is suited to large N and small M.

If M is small enough, limited by memory transfer.

If M is large, limited by GPU clock rate.

If we go all the way from time domain to frequency domain, binary tree would have depth of $log N$. In practice, we stop about halfway through.

This results in roughly equal time and frequency resolution which is what we want for useful signal processing tasks (denoising, compression, statistical learning, etc).

Summary: time complexity of our algorithm is $O(log N)$, but in practice is limited by transfer rate since $M$ is small.

## parallel dot product

product part can also be computed in parallel trivially

sum is just parallel reduce.



# Decimation filterbank

filter then downsample

!= dilation + stride (but close)

show toeplitz matrix multiplication with upsampling/downsampling

# Polyphase filterbank

http://users.ece.utexas.edu/~bevans/courses/realtime/lectures/13_Digital_PAM/lecture13.ppt

upsample then filter
 
?= transposed convolution with dilation and stride

Another oppurtunity to add CPU parallelism

show mathematically that it's equivalent to splitting filter into L pieces where L is the upsampling factor, and applying filters in parallel



# Oversampled filterbanks

(Draft) Oversampled filterbanks refers to the type of filterbank that results in larger sized output data compared to the input. For example, Fourier Transform is an oversampled filtering with factor 2, as it yields a signal in frequency domain and the phase domain. 

Because of this nature, oversampled filterbank can get better CPU parallelism by having multiple data pipeline.

# Undersampled filterbanks

(Draft) In contrast to oversampled filterbanks, undersampled filterbank results in smaller sized output data compared to the input data. An example is convolving input signal with a small kernal without paddings.

# Analysis filterbank

# Synthesis filterbank

# Processing

* denoising
* compression
* source separation

# Octave Filters + Balanced Filters

In our filter, we repeatedly apply analysis filters to break down the input signal and synthesis filters to reconstruct from broken down (and possibly processed) pieces. As shown in the earlier image, this has an unbalanced tree structure where we apply analysis filters to the higher frequency-band output at each level, but leave the lower frequency-band output. This is a natural structure, as human ear processes sound signals in a similar way. 

In our algorithm, we break down the signal into 11 octaves, as 11 is mostly sufficient to cover human’s hearing capability. As a result, we get 11 vectors, where higher frequency blocks have better resolution.  Note that, we can apply any operation on these vectors and still retain the reconstruction property, as long as the operation has the mirror-conjugate property.

Following the octave-spaced analysis filter, we apply dyadic filters in a balanced-tree fashion (i.e. apply filters to both high and low frequency outputs), to obtain some fixed frequency-resolution blocks out of the imbalanced length blocks from the previous step. 

Now, we have obtained 11 blocks of signals, decomposed from the orignal signal, where each block has the exact same frequency resolutions. We can see this as a feature representatino of the original data, where the features size is exactly same as original (i.e. critially sampled). 

Later, we will demonstrate the ease of use with this decomposition. We can perform various useful operations such as denoising and comporession with very simple code, and it is highly efficient due to massive parallelism.

# Demo our program